<a href="https://colab.research.google.com/github/ravi-ai-cons/smart_compose_sentence_completion/blob/master/Copy_of_test_smart_compose.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from tensorflow.keras.models import model_from_json
import numpy as np


In [0]:
# Connect Colab to Google Drive 
from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
with open("/content/drive/My Drive/Colab Notebooks/smart_compose_enc_model_200216_1.json", 'r') as json_file:
  enc_loaded_model_json = json_file.read()

with open("/content/drive/My Drive/Colab Notebooks/smart_compose_inf_model_200216_1.json", 'r') as json_file:
  inf_loaded_model_json = json_file.read()

enc_loaded_model = model_from_json(enc_loaded_model_json)
inf_loaded_model = model_from_json(inf_loaded_model_json)

enc_loaded_model.load_weights("/content/drive/My Drive/Colab Notebooks/smart_compose_enc_model_200216_1.h5")
inf_loaded_model.load_weights("/content/drive/My Drive/Colab Notebooks/smart_compose_inf_model_200216_1.h5")


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [0]:
class ComposeIndex():
    def __init__(self, phrase):
        self.phrase = phrase
        self.word2idx = {}
        self.idx2word = {}
        self.vocab = set()
        self.create_index()
    def create_index(self):
        for sub_phrase in self.phrase:
            self.vocab.update(sub_phrase.split(' '))
        self.vocab = sorted(self.vocab)
        self.word2idx["<pad>"] = 0
        self.idx2word[0] = "<pad>"
        for i,word in enumerate(self.vocab):
            self.word2idx[word] = i + 1
            self.idx2word[i+1] = word

In [0]:
def max_length(t):
    return max(len(i) for i in t)

# Converts the given sentence into a vector of word_Ids
# Output is 1-D: [timesteps/words]
def sentence_to_vector(sentence, phrase):
    vec = np.zeros(max_length(sentence))
    
    for i in range(1, len(sentence)):
        x_ngram = '<start> '+ sentence[:i+1] + ' <end>'
    phrase = ComposeIndex(x_ngram)
    sentence_list = [phrase.word2idx[s] for s in sentence.split(' ')]
    for i,w in enumerate(sentence_list):
        vec[i] = w
    return vec

# Given an input string, an encoder model and a decoder/inference model,
def translate(input_sentence, encoder_model, inference_model):
    sv = sentence_to_vector(input_sentence, input_sentence)
    sv = sv.reshape(1,len(sv))
    [emb_out, sh, sc] = encoder_model.predict(x=sv)
    
    start_vec = target_phrase.word2idx["<start>"]
    stop_vec = target_phrase.word2idx["<end>"]
    
    cur_vec = np.zeros((1,1))
    cur_vec[0,0] = start_vec
    cur_word = "<start>"
    output_sentence = ""
    i = 0
    while cur_word != "<end>" and i < (len_target-1):
        i += 1
        if cur_word != "<start>":
            output_sentence = output_sentence + " " + cur_word
        x_in = [cur_vec, sh, sc]
        [nvec, sh, sc] = inference_model.predict(x=x_in)
        cur_vec[0,0] = np.argmax(nvec[0,0])
        cur_word = target_phrase.idx2word[np.argmax(nvec[0,0])]
    return output_sentence

In [0]:
t = %sx read -p ''
translate(t[0].lower(), enc_loaded_model, inf_loaded_model)

KeyError: ignored